<a href="https://colab.research.google.com/github/jes000510/pub/blob/main/%5Bhw06%5D_Gender_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참여자 명단:  강은수(영어영문학과),  조재영(언어학과)

이 과제의 목적은 주어진 영어 이름의 성별을 예측하는 모형을 만드는 것이다.


# Preperation

`pandas`, `gensim` 및 `torch` 라이브러리를 가져온다.

In [ ]:
# for reading a dataset
import pandas as pd

# for pre-training an embedding
from gensim.models import FastText

# for building and training neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# Corpus

[Gender by Name](https://archive.ics.uci.edu/ml/datasets/Gender+by+Name)이라는 데이터 파일을 내려받는다.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00591/name_gender_dataset.csv

--2021-04-28 07:35:50--  https://archive.ics.uci.edu/ml/machine-learning-databases/00591/name_gender_dataset.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3774591 (3.6M) [application/x-httpd-php]
Saving to: ‘name_gender_dataset.csv’

name_gender_dataset 100%[===================>]   3.60M  6.58MB/s    in 0.5s    

2021-04-28 07:35:51 (6.58 MB/s) - ‘name_gender_dataset.csv’ saved [3774591/3774591]



`pandas`  라이브러리의 `read.csv()`  함수를 사용하면 csv 파일을 읽을 수 있다.
`DataFrame.head()`  메소드로 첫 부분을 살펴보면 Name, Gender, Count, Probability  네 개의 필드로 이루어져 있다. 데이터(Name)와 정답(Gender)이 모두 있으므로 신경망 지도 학습이 가능하다.

In [ ]:
data = pd.read_csv('name_gender_dataset.csv')
data.head()

,Name,Gender,Count,Probability
0,James,M,5304407,0.014517
1,John,M,5260831,0.014398
2,Robert,M,4970386,0.013603
3,Michael,M,4579950,0.012534
4,William,M,4226608,0.011567


각 이름을 벡터로 표현하기 위해서는 단어 임베딩 모형이 필요하다.  임베딩 모형을 훈련하기 위한 코퍼스는 문장(단어들의 리스트)들의 리스트로 표현한다. 여기에서는 이름 하나를 문장 하나로 취급한다.

In [ ]:
names = list(data['Name']) # list of names
names = [[name] for name in names] # list of lists

# Pre-training a word embedding

주어진 코퍼스로부터 단어들을 각각 100차원 벡터로 임베딩하는 `FastText` 모형을 훈련시킨다. 훈련과 관련된 하이퍼패러미터들의 값은 이후 검증 집합에서의 성능을 살펴보면서 변경할 수 있다. `help(FastText)`로 알아보라.

In [ ]:
embeddings = FastText(sentences=names, sg=1, min_count=1)

### [Q1] FastText와 Word2Vec의 공통점과 차이점을 조사해서 쓰라.  (5점)

# Building a neural network

입력층과 출력층의 차원을 결정하기 위해 단어 임베딩 벡터의 차원(`d`)과 분류할 범주의 개수(`n_classes`)를 변수로 저장한다.

In [ ]:
d = embeddings.wv.vector_size # 100-dimension
n_classes = 2  # F/M

은닉층 유닛 개수는 128로 해 보자. 이 값은 이후 검증 집합에서의 성능을 살펴보면서 변경할 수 있다.

In [ ]:
# hyperparameter setting
hidden_size = 128

입력층, 은닉층, 출력층 각 한 개로 이루어진 신경망을 구성하자.

+  `fc1`:  입력층-->은닉층. 활성화함수로 ReLU를 사용한다.
+  `fc2`: 은닉층-->출력층.  

은닉층의 개수는 이후 검증 집합에서의 성능을 살펴보면서 변경할 수 있다.

In [ ]:
class FFNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(FFNN, self).__init__()
    # input -> hidden (fully connected)
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    # hidden -> output (fully connected)
    self.fc2 = nn.Linear(hidden_dim, output_dim)
    # He initialization
    nn.init.kaiming_uniform_(self.fc1.weight)
    # Xavier initialization
    nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    # input -> hidden
    x = F.relu(self.fc1(x))
    # hidden -> output
    x = self.fc2(x)
    return x

신경망을 `net`이라는 변수로 저장한다.

In [ ]:
net = FFNN(input_dim=d, hidden_dim=hidden_size, output_dim=n_classes)
print(net)

FFNN(
  (fc1): Linear(in_features=100, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


입력 벡터를 실제로 신경망에 넣어서 출력 벡터를 뽑아 보자.

###  [Q2] 아래의 코드를 수정하여 'Cyrill'이라는 이름에 해당하는 입력 벡터를 만들라.  (1점)

In [ ]:
input = None # edit this line

입력 벡터는 100차원짜리 단어 벡터 1개로 이루어져 있으므로 100차원이 된다.

In [ ]:
print(input.size())

torch.Size([100])


분류할 범주 목록은 {F, M}  두 가지로 이루어져 있으므로 출력 벡터는 2차원이 된다.

In [ ]:
output = net(input)
print(output.size())

torch.Size([2])
tensor([-0.0405,  0.1128], grad_fn=<AddBackward0>)


# Training, validation & test datasets

언어 모형의 목적에 맞는 데이터셋을 구성하자. `torch.utils.data.Dataset` 클래스를 상속하여 새 클래스를 만들고, `__len__()` 함수와 `__getitem__()` 함수를 새로 만들면 된다.

### [Q3] 아래의 코드를 수정하여 주어진 데이터의 특성에 맞게 `__getitem__()`함수를 정의하라. 변수 `label`의 값은 여성일 때 0, 남성일 때 1이 되도록 만들라.  (2점)

In [ ]:
class NameDataset(Dataset):
  def __init__(self, data, embeddings):
    self.data = data
    self.embeddings = embeddings

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    # do something here
    embedding = None # edit this line
    label = None # edit this line
    # do something here
    sample = (embedding, label)
    return sample

위와 같은 형식으로 코퍼스에서 데이터셋을 구축한 결과 총 147269개의 데이터가 나왔다.  각 데이터는 100차원의 입력 벡터와 1개의 정답 레이블로 이루어져 있다.

In [ ]:
dataset = NameDataset(data=data, embeddings=embeddings)
print(len(dataset))
print(dataset[0])

147269
(tensor([-4.1113e-03,  8.8973e-04,  1.0328e-03, -3.3855e-03,  3.6794e-05,
         2.2390e-03, -1.8533e-03,  5.7245e-04, -2.5286e-03, -2.0550e-03,
         1.4471e-03, -2.3074e-04, -3.1000e-04,  2.4891e-04,  1.1722e-03,
         3.7925e-04, -9.8197e-05,  2.6890e-04,  7.1511e-04, -9.3246e-04,
         2.2390e-04,  1.0028e-03, -3.1352e-03, -1.9278e-03, -1.2745e-03,
        -5.5888e-04,  2.0600e-03,  7.5192e-04, -2.2592e-04, -6.9057e-04,
         9.2585e-04,  3.0719e-03,  1.3722e-04, -1.9290e-05, -1.1767e-03,
         1.5130e-03, -8.7881e-04, -8.8179e-04,  4.0520e-04,  2.0974e-03,
         2.0471e-05,  8.8442e-04,  1.6194e-03,  2.3987e-03, -1.1454e-03,
        -8.6900e-04,  1.1137e-03,  5.3693e-04,  3.6724e-03,  3.2615e-04,
         9.2433e-04, -3.4728e-03,  4.1074e-04, -2.3824e-03, -5.5873e-04,
        -8.6599e-04,  1.1831e-03, -1.8583e-04,  1.9689e-03,  6.1821e-04,
         9.0586e-04,  2.6157e-04,  1.2586e-03,  1.0602e-05, -2.2609e-04,
         1.3798e-03,  1.8795e-03, -2.7194e-

아래 코드에서는

1. 배치 사이즈를 32로 설정한다.
2. `torch.utils.data.random_split()` 함수로 훈련 집합(120000개),  검증 집합(10000개),  실험 집합(나머지)을 분할한다.
3. 각 집합을 한 번에 배치 사이즈만큼 읽어 오는 `DataLoader()` 객체를 만든다.



In [ ]:
batch_size = 32
train_dataset, valid_dataset, test_dataset = random_split(
    dataset=dataset,
    lengths=[120000, 10000, len(dataset)-130000],
    generator=torch.Generator().manual_seed(42)
    )
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Training & evaluating the neural network

##  Loss function

신경망 훈련을 위해 교차 엔트로피 손실함수를 가져온다. 실제로는 먼저 소프트맥스 활성화함수를 적용한 후에 L_CE를 계산하는 구조로 되어 있다.

In [ ]:
criterion = nn.CrossEntropyLoss()

## Optimizer

최적화기를 Adam으로 사용하고 초기 학습률을 0.01로 설정한다. 최적화기와 학습률은 이후 검증 집합에서의 성능을 살펴보면서 변경할 수 있다.

In [ ]:
learning_rate = 0.01
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## Training & Evaluating

손실함수와 최적화기를 사용하여 훈련 집합의 데이터로 신경망의 가중치 매개변수를 업데이트하고, 검증 집합에서의 정확도(accuracy)로 성능을 확인한다. epoch 횟수는 이후 검증 집합에서의 성능을 살펴보면서 변경할 수 있다.

###  [Q4]  아래의 코드를 수정하여 검증 집합에서 정확도를 계산할 수 있도록 하라.  (2점)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
  for inputs, labels in train_loader:
    # Load embeddings with gradient accumulation capabilities
    inputs = inputs.requires_grad_()

    # Clear gradients w.r.t. parameters
    optimizer.zero_grad()

    # Forward pass to get output/logits
    outputs = net(inputs)

    # Calculate Loss: softmax --> cross entropy loss
    loss = criterion(outputs, labels)

    # Getting gradients w.r.t. parameters
    loss.backward()

    # Updating parameters
    optimizer.step()

  # Calculate validation accuracy
  # do something here
  with torch.no_grad():
    for (inputs, labels) in valid_loader:
        # do something here
  accuracy = None # edit this line
  print(f'Epoch: {epoch}. Training Loss: {loss.item()}. Validation Accuracy: {accuracy}')

Epoch: 0. Training Loss: 0.7106466293334961. Validation Accuracy: 58.1
Epoch: 1. Training Loss: 0.6385602355003357. Validation Accuracy: 62.93
Epoch: 2. Training Loss: 0.6616467833518982. Validation Accuracy: 62.34
Epoch: 3. Training Loss: 0.6514435410499573. Validation Accuracy: 62.19
Epoch: 4. Training Loss: 0.6141008734703064. Validation Accuracy: 62.81
Epoch: 5. Training Loss: 0.6198220252990723. Validation Accuracy: 62.77
Epoch: 6. Training Loss: 0.6655550599098206. Validation Accuracy: 62.45
Epoch: 7. Training Loss: 0.5650405287742615. Validation Accuracy: 62.69
Epoch: 8. Training Loss: 0.6531237363815308. Validation Accuracy: 62.65
Epoch: 9. Training Loss: 0.6291529536247253. Validation Accuracy: 62.97


[Q4]와 같은 방식으로 실험 집합에서의 정확도를 계산할 수 있다.

In [ ]:
# do something here
with torch.no_grad():
  for (inputs, labels) in test_loader:
      # do something here
accuracy = None # edit this line
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 62.8589240529539


### [Q5] 위에서 언급한 여러 하이퍼패러미터를 조정해 가면서 가능한 한 높은 정확도를 달성하라.  (정확도에 10을 곱하고 소수점 셋째 자리에서 반올림한 값을 점수로 부여한다.)

# Predicting the gender

실험 집합에도 없는 새로운 이름에 대해서도 신경망이 잘 작동하는지 확인해 보자.

예를 들어 "Shrek"이라는 이름은 데이터에 없다.

In [ ]:
print(['Shrek'] in names)

False


위에서 훈련된 신경망은 이 이름을 어느 성별로 예측할까?

### [Q6] 아래의 코드를 수정하여 "Shrek"이 어느 성별로 분류되는지 예측하라. (1점)

In [ ]:
input = None # edit this line
output = net(input)
F.softmax(output, -1)

데이터에 없는 다른 이름으로 "Shica"에 대해서 같은 방식으로 확률을 구해 보자.

In [ ]:
print(['Shica'] in names)

False


### [Q7] 아래의 코드를 수정하여 "Shica"가 어느 성별로 분류되는지 예측하라.  (1점)

In [ ]:
input = None # edit this line
output = net(input)
F.softmax(output, -1)